In [31]:
# Install Pyomo
!pip install pyomo

# Install GLPK solver
!apt-get install -y glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [35]:
from pyomo.environ import *

# Example data
time_slots = range(24)  # 24 hours in a day
call_volume_forecast = [50, 40, 45, 30, 25, 20, 15, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170]  # Example forecast
cost_per_agent = 20  # $20 per agent per hour
max_hours_per_agent = 8
total_budget = 5000

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(time_slots, within=NonNegativeIntegers)  # Number of agents per time slot

# Objective function
def staffing_cost(model):
    return sum(cost_per_agent * model.x[i] for i in time_slots)

model.obj = Objective(rule=staffing_cost, sense=minimize)

# Constraints
def service_level_constraint(model, i):
    return model.x[i] >= call_volume_forecast[i]

def budget_constraint(model):
    return sum(cost_per_agent * model.x[i] for i in time_slots) <= total_budget

model.service_level = Constraint(time_slots, rule=service_level_constraint)
model.budget = Constraint(rule=budget_constraint)

# Solve the model using GLPK
solver = SolverFactory('glpk')
result = solver.solve(model, tee=True)

# Check solver status and termination condition
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found.")
    # Print results
    print("\nOptimal Number of Agents per Time Slot:")
    for i in time_slots:
        print(f"Time slot {i}: {model.x[i].value} agents")

    total_cost = sum(cost_per_agent * model.x[i].value for i in time_slots)
    print(f"\nTotal Staffing Cost: ${total_cost:.2f}")

    # Check constraints
    service_level_satisfied = all(model.x[i].value >= call_volume_forecast[i] for i in time_slots)
    budget_satisfied = total_cost <= total_budget

    print("\nConstraint Satisfaction:")
    print(f"Service Level Constraints Satisfied: {service_level_satisfied}")
    print(f"Budget Constraint Satisfied: {budget_satisfied}")

    print("\nSolver Status and Termination Condition:")
    print(f"Solver Status: {result.solver.status}")
    print(f"Termination Condition: {result.solver.termination_condition}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print("\nNo feasible solution found.")

    # Additional diagnostic information
    print("\nDiagnostic Information:")
    print("Call Volume Forecast:", call_volume_forecast)
    print("Max Hours per Agent:", max_hours_per_agent)
    print("Total Budget:", total_budget)

    # Print constraints values
    print("\nConstraints Values:")
    for i in time_slots:
        print(f"Time slot {i}:")
        print(f"  Forecast: {call_volume_forecast[i]}")
        print(f"  Required Agents: {model.x[i].value if model.x[i].value is not None else 'None'}")
        print(f"  Budget Cost for this slot: {cost_per_agent * (model.x[i].value if model.x[i].value is not None else 0)}")

    # Print all constraints
    print("\nAll Constraints:")
    for i in time_slots:
        print(f"Service Level Constraint for time slot {i}: {model.x[i].value} >= {call_volume_forecast[i]}")

    # Recalculate total cost to check if it's not calculated
    total_cost = sum(cost_per_agent * (model.x[i].value if model.x[i].value is not None else 0) for i in time_slots)
    print(f"Total Staffing Cost: ${total_cost if total_cost > 0 else 'Not Calculated'}")
else:
    print("\nSolver encountered an issue. Status: ", result.solver.status)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp8kcxax0c.glpk.raw --wglp /tmp/tmp6h6wikmc.glpk.glp --cpxlp
 /tmp/tmpgf2m9uxf.pyomo.lp
Reading problem data from '/tmp/tmpgf2m9uxf.pyomo.lp'...
25 rows, 24 columns, 48 non-zeros
24 integer variables, none of which are binary
205 lines were read
Writing problem data to '/tmp/tmp6h6wikmc.glpk.glp'...
173 lines were written
GLPK Integer Optimizer 5.0
25 rows, 24 columns, 48 non-zeros
24 integer variables, none of which are binary
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE SOLUTION
Time used:   0.0 secs
Memory used: 0.0 Mb (47725 bytes)
Writing MIP solution to '/tmp/tmp8kcxax0c.glpk.raw'...
58 lines were written

No feasible solution found.

Diagnostic Information:
Call Volume Forecast: [50, 40, 45, 30, 25, 20, 15, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170]
Max Hours per Agent: 8
Total Budget: 5000

Constraints Values:
Time slot 0:
  Forecast: 50
  Required Agent

In [36]:
!apt-get install -y -qq glpk-utils

In [60]:
num_shifts = 3  # Number of shifts in a day
time_slots_per_shift = 8
time_slots = range(num_shifts * time_slots_per_shift)  # 24 hours in a day
call_volume_forecast = [50, 40, 45, 30, 25, 20, 15, 10,
                        20, 30, 40, 50, 60, 70, 80, 90,
                        0, 10, 1, 1, 40, 50, 60, 70]  # Example forecast
cost_per_agent = 100  # $20 per agent per shift
max_agents_per_shift = 100  # Maximum number of agents allowed per shift
total_budget = 1500
income_per_call = 30  # Income generated per call answered


In [61]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(range(num_shifts), within=NonNegativeIntegers)  # Number of agents per shift

# Objective function: Maximize profit (income - cost)
def profit(model):
    total_income = sum(call_volume_forecast[i] * income_per_call for i in time_slots)
    total_cost = sum(model.x[j] * cost_per_agent for j in range(num_shifts))
    return total_income - total_cost

model.obj = Objective(rule=profit, sense=maximize)

# Constraints
def service_level_constraint(model, i):
  shift = i // time_slots_per_shift
  # Each agent handles one time slot, so the number of agents should be at least the call volume for that time slot
  # return model.x[shift] >= call_volume_forecast[i]
  return model.x[shift] >= call_volume_forecast[i] / income_per_call


def max_agents_constraint(model, j):
    return model.x[j] <= max_agents_per_shift

def budget_constraint(model):
    return sum(model.x[j] * cost_per_agent for j in range(num_shifts)) <= total_budget

model.service_level = Constraint(time_slots, rule=service_level_constraint)
model.max_agents = Constraint(range(num_shifts), rule=max_agents_constraint)
model.budget = Constraint(rule=budget_constraint)

# Solve the model using GLPK
solver = SolverFactory('glpk')
result = solver.solve(model, tee=False)

# Check solver status and termination condition
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found.")
    # Print results
    print("\nOptimal Number of Agents per Shift:")
    for j in range(num_shifts):
        print(f"Shift {j+1}: {model.x[j].value} agents")

    total_cost = sum(cost_per_agent * model.x[j].value for j in range(num_shifts))
    total_income = sum(call_volume_forecast[i] * income_per_call for i in time_slots)
    profit = total_income - total_cost
    print(f"\nTotal Staffing Cost: ${total_cost:.2f}")
    print(f"Total Income: ${total_income:.2f}")
    print(f"Profit: ${profit:.2f}")

    # Check constraints
    service_level_satisfied = all(model.x[i // time_slots_per_shift].value * time_slots_per_shift >= call_volume_forecast[i] for i in time_slots)
    budget_satisfied = total_cost <= total_budget
    max_agents_satisfied = all(model.x[j].value <= max_agents_per_shift for j in range(num_shifts))

    print("\nConstraint Satisfaction:")
    print(f"Service Level Constraints Satisfied: {service_level_satisfied}")
    print(f"Budget Constraint Satisfied: {budget_satisfied}")
    print(f"Max Agents per Shift Constraints Satisfied: {max_agents_satisfied}")

    print("\nSolver Status and Termination Condition:")
    print(f"Solver Status: {result.solver.status}")
    print(f"Termination Condition: {result.solver.termination_condition}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print("\nNo feasible solution found.")
else:
    print("\nSolver encountered an issue. Status: ", result.solver.status)



Optimal solution found.

Optimal Number of Agents per Shift:
Shift 1: 2.0 agents
Shift 2: 3.0 agents
Shift 3: 3.0 agents

Total Staffing Cost: $800.00
Total Income: $27210.00
Profit: $26410.00

Constraint Satisfaction:
Service Level Constraints Satisfied: False
Budget Constraint Satisfied: True
Max Agents per Shift Constraints Satisfied: True

Solver Status and Termination Condition:
Solver Status: ok
Termination Condition: optimal


Optimal Number of Agents per Shift:
- Shift 1: 5.0 agents
- Shift 2: 9.0 agents
- Shift 3: 7.0 agents

- Total Staffing Cost: $1050.00

- Total Income: $9070.00

- Profit: $8020.00


Optimal Number of Agents per Shift:
- Shift 1: 2.0 agents
- Shift 2: 3.0 agents
- Shift 3: 3.0 agents

- Total Staffing Cost: $800.00

- Total Income: $27210.00

- Profit: $26410.00
